# Empty Response Test

In [ ]:
import pytest
from unittest.mock import AsyncMock, patch, MagicMock
from uuid import uuid4

from dotenv import load_dotenv
import django

In [ ]:
load_dotenv("../tests/.env.test")
django.setup()

In [ ]:
from redbox_app.redbox_core.consumers import ChatConsumer

In [ ]:
@pytest.mark.asyncio
async def test_llm_conversation_with_empty_response():
    consumer = ChatConsumer(scope={"user", MagicMock(id=uuid4())})

    # Mock Chat and Files
    mock_session = MagicMock()
    mock_user = MagicMock(id=uuid4())
    mock_file = MagicMock(unique_name="file1")
    selected_files = [mock_file]
    permitted_files = [mock_file]

    # Mock Chat Messages
    chat_message1 = MagicMock(role="user", text="Hello")
    chat_message2 = MagicMock(role="ai", text="")  # Blank
    # replaced django_app.redbox_app.redbox_core.models.ChatMessage by redbox_app.redbox_core.models.ChatMessage
    with (
        patch("redbox_app.redbox_core.models.ChatMessage") as mock_filter,
        patch.object(consumer, "get_ai_settings", new_callable=AsyncMock) as mock_get_ai_settings,
        patch.object(consumer.redbox, "run", new_callable=AsyncMock) as mock_redbox_run,
        patch.object(consumer, "send_to_client", new_callable=AsyncMock) as mock_send_to_client,
        patch.object(consumer, "save_ai_message", new_callable=AsyncMock),
    ):
        mock_filter.return_value.order_by.return_value.__aiter__.return_value = [chat_message1, chat_message2]
        mock_get_ai_settings.return_value = {}
        mock_redbox_run.return_value = {}

        with pytest.raises(ValueError, match="Null LLM Response Received"):
            await consumer.llm_conversation(selected_files, mock_session, mock_user, "Test Title", permitted_files)
        mock_send_to_client.assert_called()

In [ ]:
await test_llm_conversation_with_empty_response()